In [1]:
'''import all necessary libraries'''

import cx_Oracle #import the oracle database libraries
import pandas as pd #connect to the pandas libraries
import numpy as np #import numpy
import getpass #import the password libraries in order to keep database password concealed

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
import sys
print (sys.version)
pd.options.display.float_format = '{:0.6f}'.format

3.6.5 |Anaconda custom (64-bit)| (default, Mar 29 2018, 13:32:41) [MSC v.1900 64 bit (AMD64)]


In [2]:
'''connect to the oracle server used to pull down the 
currency and misuse of authorization information'''

username=''
password=''
#password=getpass.getpass("Enter your password: ")

con=cx_Oracle.connect(username,password,'ODSPEU1.ams') #enter database details
print (con.version) # prnt the conversion details

12.1.0.2.0


In [3]:
bnp=pd.read_sql(
''' 
select Acquirer,PRODUCT_NAME,BN.TRANSACTION_TYPE,BN.CREDIT_DEBIT,BN.CONS_CORP,SUM(BN.TRANSACTION_FLOW_EUR) AS TRANSACTION_FLOW_EUR
from FDWO.ACQUIRER_BNP_COST bn
cross join
(select 'BNP' as Acquirer from dual)
left join
(select PRODUCT_NAME,PRODUCT_CODE from FDWO.MB_PRODUCT_CODE) pc
on bn.PRODUCT_CODE=PC.PRODUCT_CODE
where BN.CONTRACT_ID in('8423','8535')
and BN.TRANSACTION_DATE between to_date('10/1/2018','MM/DD/YYYY') and to_date('12/31/2018','MM/DD/YYYY')
group by BN.TRANSACTION_TYPE,BN.CREDIT_DEBIT,BN.CONS_CORP,Acquirer,PRODUCT_NAME
order by TRANSACTION_FLOW_EUR desc
''', con)

In [6]:
elavon=pd.read_sql(
''' 
select Acquirer,CARD_SCHEME,CARD_TYPE,CARD_DESC,INTERCHANGE_DESC,INTERCH_PERC,INTERCH_FIXED,PRODUCT_NAME,TRANSACTION_TYPE,FLOW_EUR,CARD_CATEGORY
from
(
    select Acquirer,EV.CARD_SCHEME,EV.CARD_TYPE,EV.CARD_DESC,EV.INTERCHANGE_DESC,EV.INTERCH_PERC,EV.INTERCH_FIXED,PRODUCT_NAME,EV.TRANSACTION_TYPE,SUM(EV.FLOW_EUR) AS FLOW_EUR
    ,case when CARD_DESC in ('VICONSUMER','M/C CONSUM') then 'Consumer'
    when CARD_DESC in ('VI COMMER','M/C COMMER') then 'Commercial'  
    when CARD_DESC = 'VISA SIGN' then 'Signature'
    when CARD_DESC in ('VI DEBIT','M/C DEBIT') then 'Debit'
    when CARD_DESC in ('WORLD CARD') then 'World'
    when CARD_DESC in ('M/C FLEET') then 'Fleet'
    when CARD_DESC in ('M/C PURCH') then 'Purchase'
    end as CARD_CATEGORY
    from FDWO.ACQUIRER_ELAVON_USA_COST ev
    cross join
    (select 'Elavon' as Acquirer from dual)
    left join
    (select PRODUCT_NAME,PRODUCT_CODE from FDWO.MB_PRODUCT_CODE) pc
    on ev.PRODUCT_CODE=PC.PRODUCT_CODE
    where EV.MID_NAME like '%AIR TRAN%'
    and EV.TRANS_DATE between to_date('10/1/2018','MM/DD/YYYY') and to_date('12/31/2018','MM/DD/YYYY')
    and EV.CARD_SCHEME not in ('VIDE')
    group by Acquirer,EV.CARD_SCHEME,EV.CARD_TYPE,EV.CARD_DESC,EV.INTERCHANGE_DESC,EV.INTERCHANGE_DESC,EV.INTERCH_PERC,EV.INTERCH_FIXED,EV.TRANSACTION_TYPE,PRODUCT_NAME
    order by FLOW_EUR desc
)
UNION ALL
select Acquirer,CARD_SCHEME,CARD_TYPE,CARD_DESC,INTERCHANGE_DESC,INTERCH_PERC,INTERCH_FIXED,PRODUCT_NAME,TRANSACTION_TYPE,FLOW_EUR,CARD_CATEGORY
from
(
    select Acquirer,EV.CARD_SCHEME,EV.CARD_TYPE,EV.CARD_DESC,EV.INTERCHANGE_DESC,EV.INTERCH_PERC,EV.INTERCH_FIXED,PRODUCT_NAME,EV.TRANSACTION_TYPE,SUM(FLOW_EUR) AS FLOW_EUR
    ,case when CARD_DESC in ('VICONSUMER','M/C CONSUM') then 'Consumer'
    when CARD_DESC in ('VI COMMER','M/C COMMER') then 'Commercial'  
    when CARD_DESC = 'VISA SIGN' then 'Signature'
    when CARD_DESC in ('VI DEBIT','M/C DEBIT') then 'Debit'
    when CARD_DESC in ('WORLD CARD') then 'World'
    when CARD_DESC in ('M/C FLEET') then 'Fleet'
    when CARD_DESC in ('M/C PURCH') then 'Purchase'
    end as CARD_CATEGORY
    from FDWO.ACQUIRER_ELAVON_USA_COST ev
    cross join
    (select 'Elavon' as Acquirer from dual)
    left join
    (select PRODUCT_NAME,PRODUCT_CODE from FDWO.MB_PRODUCT_CODE) pc
    on ev.PRODUCT_CODE=PC.PRODUCT_CODE
    where EV.MID_NAME like '%AIR TRAN%'
    and EV.TRANS_DATE between to_date('10/1/2018','MM/DD/YYYY') and to_date('12/31/2018','MM/DD/YYYY')
    and EV.TRANSACTION_TYPE='CHARGEBACK'
    group by Acquirer,EV.CARD_SCHEME,EV.CARD_TYPE,EV.CARD_DESC,EV.INTERCHANGE_DESC,EV.INTERCH_PERC,EV.INTERCH_FIXED,PRODUCT_NAME,EV.TRANSACTION_TYPE
)
order by FLOW_EUR desc
''', con)

In [8]:
aib=pd.read_sql(
''' 
select Acquirer,AB.TRANSACTION_TYPE,AB.CARD_BRAND,AB.CARD_TYPE_GROUP,AB.CARD_TYPE_CODE,AB.CARD_TYPE
,PC.PRODUCT_NAME,SUM(AB.TRANSACTION_FLOW_EUR) as TRANSACTION_FLOW_EUR
,case when CARD_TYPE in ('Mastercard Black') then 'Black'
when CARD_TYPE in ('MasterCard B2B Product','Visa Business') then 'Business'  
when CARD_TYPE in ('Mastercard Or Eurocard Standard','Mastercard Or Eurocard Card - Standard','Visa Classic','MasterCard Standard Deferred','MasterCard standard - unspecified detail','Visa Standard - unspecified detail') then 'Consumer'
when CARD_TYPE in ('Eurocard-Mastercard Corporate Card','Visa Corporate T and E','Mastercard Or Eurocard Corporate-Businesscard') then 'Corporate'
when CARD_TYPE in ('Mastercard Or Eurocard Corporate-Businesscard','Visa Classic Debit','Visa Gold Debit','Mastercard Debit Standard Card','Visa Platinum Debit','Visa Business Debit'
                    ,'Gold Mastercard Card-Immediate Debit','Mastercard Standard Card-Immediate Debit','Visa Purchasing Deferred Debit','Visa Classic Deferred Debit','MasterCard World Signia Card Immediate Debit'
                    ,'Visa Corporate T and E Debit','Visa Electron Debit','Visa Business Deferred Debit','Mastercard Debit Platinum','Mastercard Debit Other 2 Embossed','MasterCard Debit Unembossed','Visa Corporate T and E Deferred Debit'
                    ,'MasterCard Immediate Debit ICIS Student') then 'Debit'
when CARD_TYPE in ('Mastercard Or Eurocard Corporate Fleet Card') then 'Fleet'
when CARD_TYPE in ('Visa Gold','Mastercard Or Eurocard Gold Card') then 'Gold'
when CARD_TYPE in ('Visa High Net Worth HNW Consumer - Canada only') then 'HNW'  
when CARD_TYPE in ('Visa Infinite','Visa Infinite Prepaid') then 'Infinite'  
when CARD_TYPE in ('Mastercard New World Card') then 'New World'
when CARD_TYPE in ('Visa Platinum','Mastercard Or Eurocard Platinum Card') then 'Platinum'
when CARD_TYPE in ('Visa Electron Prepaid','Mastercard Prepaid Non-US','MasterCard Prepaid business card','Mastercard Prepaid General Spend'
                    ,'Visa Platinum Prepaid','Visa Classic Prepaid','Mastercard Prepaid Unembossed') then 'Prepaid'
when CARD_TYPE in ('Visa Purchasing') then 'Purchasing'
when CARD_TYPE in ('Mastercard Rewards Only') then 'Rewards'
when CARD_TYPE in ('Visa Select') then 'Select'  
when CARD_TYPE in ('Visa Signature Preferred','Visa Signature') then 'Signature'  
when CARD_TYPE in ('Titanium Mastercard') then 'Titanium'
when CARD_TYPE in ('Visa Traditional') then 'Traditional'
when CARD_TYPE in ('Visa Traditional Rewards') then 'Traditional rewards'
when CARD_TYPE in ('Mastercard World Elite Card') then 'World Elite'
when CARD_TYPE in ('Mastercard Or Eurocard World Signia Card') then 'World Signia'
end as CARD_CATEGORY
from FDWO.ACQUIRER_AIB_NEWGEN_FILES ab
cross join
(select 'AIB' as Acquirer from dual)
left join
(select PRODUCT_NAME,PRODUCT_CODE from FDWO.MB_PRODUCT_CODE) pc
on AB.PRODUCT_CODE=PC.PRODUCT_CODE
where AB.TRADING_MERCHANT_NAME like '%Air Trans%'
and AB.TRANSACTION_DATE between to_date('10/1/2018','MM/DD/YYYY') and to_date('12/31/2018','MM/DD/YYYY')
and AB.PROCESSING_STATUS not in ('Suspended')
group by Acquirer,AB.TRANSACTION_TYPE,AB.CARD_BRAND,AB.CARD_TYPE_GROUP,AB.CARD_TYPE_CODE,AB.CARD_TYPE
,PC.PRODUCT_NAME
order by TRANSACTION_FLOW_EUR desc
''', con)

In [11]:
with pd.ExcelWriter('d:\\djohnson\\Desktop\\air_transat.xlsx') as writer:
    aib.to_excel(writer, sheet_name='AIB')
    bnp.to_excel(writer, sheet_name='BNP')
    elavon.to_excel(writer, sheet_name='ELAVON')    